In [6]:
from __future__ import division
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [7]:
import os
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize']=8,5

In [8]:
#import data
os.chdir("/home/gladkov/storage/al_R/3new")
os.listdir()
otus = pd.read_csv("otu_table_for_rf_merged.txt", sep="\t")
metadata = pd.read_csv("metadata_for_rf_merged.txt", sep="\t")



In [9]:
# some function for more suitable rownames
def rownames_creator(x):
    first_column = x[0]
    repeats_column = x[2]
    repeat_number = first_column.split(".")[1]
    rownames = repeats_column + "." + repeat_number
    return rownames
# return first_column
metadata["rownames"] = metadata.apply(rownames_creator, axis=1, result_type="expand")
metadata = metadata.rename({metadata.columns[0]: "rowname_old"}, axis="columns")
metadata = metadata.set_index("rownames")
meta_col = list(metadata.columns)

# repair otu table & create nicy rownames like above
# otus.columns = otus.iloc[0]
# otus = otus.drop(["Unnamed: 0"])
# not any need to repair?
# otus.index = meta_col
meta_index = list(metadata.index)
otus.index = meta_index
otus = otus.drop(columns=["Unnamed: 0"])

IndexError: ('list index out of range', 'occurred at index 0')

In [12]:
# same shit that above with merged repeats
metadata.index = metadata["Unnamed: 0"]
metadata = metadata.drop(columns=["Unnamed: 0"])

otus.index = otus["Unnamed: 0"]
otus = otus.drop(columns=["Unnamed: 0"])

metadata = metadata.drop(columns=["Filename","Description","Plant","Inoculation","Al", "QRTPCR"])

KeyError: 'Unnamed: 0'

In [10]:
#create df with factor that consist only numerical data
met_col_list = list(metadata.columns)
count_list = list(range(0,105))
# np.column_stack((count_list, met_col_list))
# metadata = metadata.drop([])


In [5]:
metadata

,Unnamed: 0,Filename,Description,Plant,Inoculation,Al,BineM,SeedM,BineSeedM,SeedN,...,BineSeedPerPlantFe,BineSeedPerPlantK,BineSeedPerPlantMg,BineSeedPerPlantMo,BineSeedPerPlantNi,BineSeedPerPlantP,BineSeedPerPlantS,BineSeedPerPlantS.1,BineSeedPerPlantZn,QRTPCR
0,s1903,26.5,1,1,1,1,5.217500,4.342500,9.1975,40.5,...,766.378262,714.277514,62.535185,76.387623,147.119788,80.156457,189.861096,47.650990,162.618635,1592
1,s1903_Al,59.5,2,1,1,2,4.392500,3.815000,8.3575,32.5,...,666.361386,693.159352,55.043641,73.033851,296.895381,20.194510,163.559044,40.544464,152.699049,2426
2,s1903_In,2.5,3,1,2,1,5.927500,5.102500,11.0300,38.5,...,1020.152172,756.427953,81.277633,90.928284,186.786847,113.749773,218.738423,59.010729,317.804489,1546
3,s1903_In_Al,5.0,4,1,2,2,6.280000,4.510000,10.7600,38.0,...,882.726277,1024.193155,85.113614,102.978868,432.093049,31.304417,204.789880,69.358599,197.320157,6881
4,s7307,8.5,5,2,1,1,6.171667,9.320833,15.7325,37.0,...,1228.812499,610.736871,79.297033,118.448587,197.922382,54.625197,346.483515,50.242766,314.062153,6980
5,s7307_Al,13.5,6,2,1,2,5.790000,6.465000,12.2875,29.5,...,1348.291963,814.213264,82.019465,193.789526,511.311272,30.422035,316.907947,56.710160,514.203436,8322
6,s7307_In,17.5,7,2,2,1,7.550000,9.617500,17.2850,36.0,...,1716.719807,964.578965,93.713259,165.309027,283.260445,84.401096,422.719562,70.984900,450.168242,21605
7,s7307_In_Al,22.5,8,2,2,2,6.807500,6.432500,13.3475,28.5,...,1067.851311,831.875468,92.289192,179.595986,565.965374,30.928963,298.532203,62.610131,426.562850,247735
8,s8353,43.5,9,3,1,1,3.290000,4.607500,7.8975,19.5,...,642.869897,450.987282,45.250730,112.997601,219.508415,96.767008,264.529665,50.103618,203.462065,1151000
9,s8353_Al,47.5,10,3,1,2,2.430000,3.932500,6.4175,15.5,...,909.982382,432.819014,37.856930,104.608591,303.405715,24.295080,224.481853,39.265409,253.188558,1437000


In [12]:
otus

,Seq9032,Seq9292,Seq9026,Seq9514,Seq6744,Seq9467,Seq9390,Seq9750,Seq4443,Seq8153,...,Seq4725,Seq2306,Seq7279,Seq8112,Seq5958,Seq5255,Seq8827,Seq8848,Seq9482,Seq8604
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
s1903,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,13.019559,12.381450,...,11.710072,13.380344,11.710072,12.906451,11.710072,11.710072,11.710072,11.710072,11.710072,12.885218
s1903_Al,11.710072,11.710072,11.710072,11.710072,12.657625,11.710072,11.710072,11.710072,11.710072,11.710072,...,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,13.028021,11.710072,11.710072,11.710072
s1903_In,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,13.167133,12.654322,...,12.765952,13.618598,11.710072,12.246774,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072
s1903_In_Al,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,...,9.706624,9.322083,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554,8.782554
s7307,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,12.177178,...,11.710072,13.543454,11.710072,11.710072,11.710072,11.710072,12.237664,11.710072,11.710072,11.710072
s7307_Al,11.710072,11.710072,11.710072,11.710072,11.710072,12.284653,11.710072,11.710072,11.710072,12.739978,...,11.710072,14.804255,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072
s7307_In,11.710072,11.710072,11.710072,11.710072,12.914762,11.710072,11.710072,11.710072,11.710072,11.710072,...,14.324443,13.400823,13.829970,12.212878,12.431779,11.710072,11.710072,12.709640,12.930595,12.701177
s7307_In_Al,11.710072,11.710072,12.564065,11.710072,11.710072,11.710072,11.710072,11.710072,13.926717,12.207882,...,13.376533,12.780527,11.710072,13.376397,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072
s8353,11.710072,12.918492,11.710072,11.710072,11.710072,12.427628,11.710072,12.471904,12.929170,12.231530,...,11.710072,15.326957,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072,11.710072


In [188]:
rf = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)
# rf(otus, metadata)
X_train, X_valid, y_train, y_valid = train_test_split(otus, metadata, test_size = 0.8, random_state = 42)
rf.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [11]:
count(metadata["Al"])

NameError: name 'count' is not defined

In [59]:
BineMsumByAl = metadata \
.groupby(["Plant", "Al"]) \
.agg({"BineM":"sum", "Plant": "count"}) \
.rename(columns={"BineM":"SumBineM", "Plant":"PlantCount"}) \
.sort_values("SumBineM", ascending=False)
    

In [59]:
BineMsumByAl = metadata \
.groupby(["Plant", "Al"]) \
.agg({"BineM":"sum", "Plant": "count"}) \
.rename(columns={"BineM":"SumBineM", "Plant":"PlantCount"}) \
.sort_values("SumBineM", ascending=False)
    

In [39]:
if int(BineMsumByAl.SumBineM.sum()) == int(metadata.BineM.sum()):
    print("OK")
else:
    print("ERRoR")

OK


In [36]:
metadata.BineM.sum()

70.26916666666668

In [60]:
BineMsumByAl

SumBineM  PlantCount
Plant Al                       
2     1   13.721667           2
      2   12.597500           2
1     1   11.145000           2
      2   10.672500           2
3     1    6.600000           2
4     1    5.672500           2
3     2    5.122500           2
4     2    4.737500           2